the package installation steps are for execution on collab

In [10]:
%autosave 300
%autoreload 2
%reload_ext autoreload
%config Completer.use_jedi = False

Autosaving every 300 seconds


In [11]:
# !pip install portalocker
# !pip install torchview
# !pip install torcheval
# !pip install scikit-plot
# !pip install lime
# !pip install -q transformers datasets

In [12]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchtext
from torch.utils.data import DataLoader, TensorDataset
from torchtext import data
from torchtext import datasets
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import re
from torch.utils.data import DataLoader
from torchtext.data.functional import to_map_style_dataset
from torchsummary import summary
from torchview import draw_graph
import numpy as np
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
from torcheval.metrics import MulticlassAccuracy,BinaryAccuracy
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import confusion_matrix
import scikitplot as skplt
import spacy

In [14]:
def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

In [15]:
# Set manual seed since nn.Parameter are randomly initialzied
set_seed(42)
# Set device cuda for GPU if it's available otherwise run on the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
batch_size = 1024
epochs = 10
lr = 1e-4
embedding = False

Random seed set as 42
cuda


## Tokenizer

In [16]:
# Using custom dataset from the torchtext module
train, valid, test = datasets.PennTreebank()

In [19]:
# spacy.cli.download('en_core_web_sm')
spacy_en = spacy.load("en_core_web_sm")
def spacy_tokenize(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

/home/azureuser/.local/lib/python3.10/site-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [21]:
# toch text tokenizer
tokenizer = get_tokenizer("basic_english")

In [25]:
tokenizer("I have a dog!!")

['i', 'have', 'a', 'dog', '!', '!']

In [27]:
spacy_tokenize("I have a dog!!")

['I', 'have', 'a', 'dog', '!', '!']

## Vocabulary